In [1]:
from datasets import load_dataset

import torch
from torchvision import transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np


import sys
import os
# %load_ext autoreload
# %autoreload 2

In [3]:
ds = load_dataset("MKZuziak/ISIC_2019_224", split="train") # que le split train disponible

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

data.zip:   0%|          | 0.00/143M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [4]:
ds_small = ds.train_test_split(train_size=0.1, seed=42)["train"]

ds_split = ds_small.train_test_split(test_size=0.2, seed=42)

train_ds = ds_split["train"]
test_ds = ds_split["test"]

In [5]:
# On ajoute le dossier au chemin de recherche
!git clone https://github.com/colinhl2002/DL-Dense-Object-Detection.git

Cloning into 'DL-Dense-Object-Detection'...
remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 19 (delta 5), reused 16 (delta 4), pack-reused 0 (from 0)
Receiving objects: 100% (19/19), 129.10 KiB | 189.00 KiB/s, done.
Resolving deltas: 100% (5/5), done.


In [6]:
sys.path.append('DL-Dense-Object-Detection')

In [7]:
from utils import ISICDataset

my_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# On passe le dataset chargé précédemment à notre classe
custom_ds = ISICDataset(train_ds, transform=my_transform)

In [8]:
len(custom_ds)

2026

In [9]:
from utils import ISICDataset

my_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# On passe le dataset chargé précédemment à notre classe
custom_ds = ISICDataset(train_ds, transform=my_transform)

train_loader = DataLoader(
    custom_ds,
    batch_size=16,
    shuffle=True, # Important pour le déséquilibre
)

# --- ÉTAPE 3 : Affichage des 16 premières images ---
# On récupère un batch

batch = next(iter(train_loader))
images, labels = batch

# plt.figure(figsize=(8, 8))
# for i in range(16):
#     plt.subplot(4, 4, i + 1)
#     # PyTorch utilise (C, H, W), Matplotlib attend (H, W, C) -> on utilise permute
#     img_display = images[i].permute(1, 2, 0).numpy()
#     plt.imshow(img_display)
#     plt.title(f"Classe: {labels[i].item()}")
#     plt.axis("off")

# plt.tight_layout()
# plt.show()

In [10]:
from model import cnn_model
from train import setup_training, train_one_epoch

In [ ]:
# Init the training
model = cnn_model(num_classes=8)
criterion, optimizer = setup_training(model, learning_rate=0.01, momentum=0.9)

DEVICE = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
print(DEVICE)
model = model.to(DEVICE)
# print(model)

train_losses = []
valid_losses = []

EPOCHS = 10
# loop over the dataset multiple times
for epoch in range(EPOCHS):
    model.train()
    train_epoch_loss = train_one_epoch(model, criterion, optimizer, train_loader, DEVICE)
    # model.eval()
    # valid_epoch_loss = valid_one_epoch(model, val_loader)

    print(f"EPOCH={epoch}, TRAIN={train_epoch_loss:.05f}")

    train_losses.append(train_epoch_loss)
    # valid_losses.append(valid_epoch_loss)

cuda:0


 67%|██████▋   | 85/127 [06:00<02:59,  4.28s/it]

In [ ]:
# State dict saving
with open("model.pt", "wb") as f:
    torch.save(model.state_dict(), f)

In [ ]:
from tqdm import tqdm

y_true = []
y_pred = []

# Send model to correct device
model.to(DEVICE)

# Put model in evaluatio mode (very important)
model.eval()

# Disable all gradients things
with torch.no_grad():
    for x, y_t in tqdm.tqdm(test_ds, "predicting"):
        x = x.reshape((-1,) + x.shape)
        x = x.to(DEVICE)
        y = model.forward(x)
        y = y.to("cpu").numpy()

        y_t = int(y_t.to("cpu").numpy())

        y_pred.append(y)
        y_true.append(y_t)


y_pred = np.concatenate(y_pred, axis=0)
y_true = np.asarray(y_true)

y_pred_classes = y_pred[:, 0] > 0.5